In [1]:
import numpy as np
import nn
import idx2numpy

In [2]:
train_img = (idx2numpy.convert_from_file("train-images.idx3-ubyte")/255).reshape(-1, 28 * 28)
train_lbl = idx2numpy.convert_from_file("train-labels.idx1-ubyte")
test_img = (idx2numpy.convert_from_file("t10k-images.idx3-ubyte")/255).reshape(-1, 28 * 28)
test_lbl = idx2numpy.convert_from_file("t10k-labels.idx1-ubyte")

In [3]:
train_img.shape, test_img.shape

((60000, 784), (10000, 784))

In [4]:
train_lbl.shape

(60000,)

In [10]:
l1 = nn.Layer_Dense(28*28, 123)
a1 = nn.Activation_ReLU()
l2 = nn.Layer_Dense(123, 123)
a2 = nn.Activation_ReLU()
l3 = nn.Layer_Dense(123, 10)
a3 = nn.Activation_Softmax()
loss = nn.Loss_CategoricalCrossentropy()
optim = nn.Optimizer_Adam()

In [11]:
for i in range(15):
    l1.forward(train_img)
    a1.forward(l1.output)
    l2.forward(a1.output)
    a2.forward(l2.output)
    l3.forward(a2.output)
    a3.forward(l3.output)
    print(f'Epoch = {i + 1}  Loss = {loss.calculate(a3.output, train_lbl)}  Acc = {a3.prediction()}')
    loss.backward(a3.output, train_lbl)
    a3.backward(loss.dinputs)
    l3.backward(a3.dinputs)
    a2.backward(l3.dinputs)
    l2.backward(a2.dinputs)
    a1.backward(l2.dinputs)
    l1.backward(a1.dinputs)
    optim.pre_update_params()
    optim.update_params(l1)
    optim.update_params(l2)
    optim.update_params(l3)
    optim.post_update_params()


Epoch = 1  Loss = 2.40420079333165  Acc = 0.0
Epoch = 2  Loss = 2.2701353045561605  Acc = 0.0
Epoch = 3  Loss = 2.154449627888097  Acc = 0.0
Epoch = 4  Loss = 2.0501429857739484  Acc = 0.0
Epoch = 5  Loss = 1.952448019030373  Acc = 0.0
Epoch = 6  Loss = 1.8583747703165328  Acc = 0.0
Epoch = 7  Loss = 1.7660927885206053  Acc = 0.0
Epoch = 8  Loss = 1.6749558826892905  Acc = 0.0
Epoch = 9  Loss = 1.5847290477244216  Acc = 0.0
Epoch = 10  Loss = 1.4954783961381382  Acc = 0.0
Epoch = 11  Loss = 1.4074136673924569  Acc = 0.0
Epoch = 12  Loss = 1.3209851176035836  Acc = 0.0
Epoch = 13  Loss = 1.2368222195850098  Acc = 0.0
Epoch = 14  Loss = 1.1557134507970708  Acc = 0.0
Epoch = 15  Loss = 1.0784523027806394  Acc = 0.0
